In [0]:
pip install -q pyyaml h5py

In [0]:
import tensorflow as tf
from tensorflow import keras
import os

In [3]:
(training_images, training_labels), (testing_images, testing_labels) = tf.keras.datasets.mnist.load_data()
training_labels = training_labels[:1000]
testing_labels = testing_labels[:1000]
training_images = training_images[:1000].reshape(-1, 28 * 28) / 255.0
testing_images = testing_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def build_model():
  m = tf.keras.models.Sequential([ keras.layers.Dense(512, activation='relu', input_shape=(784,)), keras.layers.Dropout(0.2), keras.layers.Dense(10)])
  m.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  return m

In [8]:
m = build_model()
m.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
cp_path = "training_1/cp.ckpt"
cp_dir = os.path.dirname(cp_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, save_weights_only=True, verbose=1)
m.fit(training_images, training_labels, epochs=10, validation_data=(testing_images,testing_labels), callbacks=[cp_callback]) 

Epoch 1/10
27/32 [========================>.....] - ETA: 0s - loss: 1.2263 - accuracy: 0.6597
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 1.1452 - accuracy: 0.6820 - val_loss: 0.6891 - val_accuracy: 0.7930
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.4142 - accuracy: 0.8774
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4335 - accuracy: 0.8700 - val_loss: 0.5474 - val_accuracy: 0.8240
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2805 - accuracy: 0.9213
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2932 - accuracy: 0.9220 - val_loss: 0.4840 - val_accuracy: 0.8500
Epoch 4/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2074 - accuracy: 0.9483
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [11]:
ls {cp_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [12]:
m = build_model()
loss, acc = m.evaluate(testing_images,  testing_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3628 - accuracy: 0.0880
Untrained model, accuracy:  8.80%


In [13]:
m.load_weights(cp_path)
loss,acc = m.evaluate(testing_images,  testing_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4008 - accuracy: 0.8720
Restored model, accuracy: 87.20%


In [14]:
cp_path = "training_2/cp-{epoch:04d}.ckpt"
cp_dir = os.path.dirname(cp_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, verbose=1, save_weights_only=True, period=5)

m = build_model()
m.save_weights(cp_path.format(epoch=0))
m.fit(training_images, training_labels, epochs=50, callbacks=[cp_callback], validation_data=(testing_images,testing_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [15]:
ls {cp_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [16]:
latest = tf.train.latest_checkpoint(cp_dir)
latest

'training_2/cp-0050.ckpt'

In [18]:
m = build_model()
m.load_weights(latest)

loss, acc = m.evaluate(testing_images,  testing_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4749 - accuracy: 0.8800
Restored model, accuracy: 88.00%


In [20]:
m.save_weights('./checkpoints/my_checkpoint')

m = build_model()
m.load_weights('./checkpoints/my_checkpoint')
loss,acc = m.evaluate(testing_images,  testing_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4749 - accuracy: 0.8800
Restored model, accuracy: 88.00%


In [23]:
m = build_model()
m.fit(training_images, training_labels, epochs=5)

!mkdir -p saved_model
m.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.2246 - accuracy: 0.6250
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4211 - accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2911 - accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2138 - accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1486 - accuracy: 0.9660
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [24]:
!ls saved_model
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [25]:
new_m = tf.keras.models.load_model('saved_model/my_model')
# Check its architecture
new_m.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_m.evaluate(testing_images,  testing_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))
print(new_m.predict(testing_images).shape)

32/32 - 0s - loss: 0.4599 - accuracy: 0.8530
Restored model, accuracy: 85.30%
(1000, 10)


In [28]:
m = build_model()
m.fit(training_images, training_labels, epochs=5)
m.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1296 - accuracy: 0.6790
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4125 - accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2904 - accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2170 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1455 - accuracy: 0.9700


In [29]:
new_m = tf.keras.models.load_model('my_model.h5')
new_m.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
loss, acc = new_m.evaluate(testing_images,  testing_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.3988 - accuracy: 0.8680
Restored model, accuracy: 86.80%
